In [2]:
# 导入所需要的库
import mindspore
import mindspore.nn as nn
from mindspore.common.initializer import Normal
import mindspore.dataset as ds
from mindspore.train.callback import LossMonitor
# import mindspore.dataset.vision as vision
import mindspore.dataset.vision.py_transforms as vision
# import mindspore.dataset.transforms as transforms
from mindspore.dataset.transforms import c_transforms as transforms
from mindspore import dtype as mstype
from mindspore.nn import Accuracy
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
from mindspore import context
from PIL import Image
import numpy as np
import mindspore.ops as ops
from mindspore.train.serialization import load_checkpoint, load_param_into_net


# 加载数据集函数
def load_dataset(dataset_path, batch_size=24, repeat_num=1):
    # 定义参数
    image_size = [100, 100]
    # mean = [127.5]
    # std = [255]
    # 构建标签索引
    class_indexing = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9}
    dataset = ds.ImageFolderDataset(dataset_path, shuffle=True, decode=True, class_indexing=class_indexing)  # 打乱数据集，并编码
    trans = [
        vision.ToPIL(),
        vision.Grayscale(),
        vision.Resize(image_size),
        # vision.Normalize(mean=mean, std=std),
        vision.ToTensor()
        # vision.HWC2CHW()
    ]
    type_cast_op = transforms.TypeCast(mstype.int32)
    # 将数据集转为一维并规范尺寸
    dataset = dataset.map(operations=trans, input_columns="image", num_parallel_workers=8)
    # 改变标签的数据类型
    dataset = dataset.map(operations=type_cast_op, input_columns="label", num_parallel_workers=8)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(repeat_num)
    return dataset


# 定义网络结构
class my_net(nn.Cell):
    def __init__(self):
        super(my_net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5,
                               pad_mode='valid')  # 2维卷积 参数：（input_channel , output_channel , kernel_size , pad_mode）
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.relu = nn.ReLU()  # 线性激活函数 ReLU(x)=max(0,x),
        self.max_pool2d = nn.MaxPool2d(kernel_size=4, stride=2)  # 池化层
        self.flatten = nn.Flatten()  # 展平张量而不改变第 0 轴上批量大小的维度。
        self.fc1 = nn.Dense(16 * 20 * 20, 3200, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(3200, 1000, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(1000, 10, weight_init=Normal(0.02))

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# 加载test集函数
def load_testdata(testdata_path):
    image = Image.open(testdata_path).convert("F")
    image_size = (100, 100)
    image = image.resize(image_size)
    image = np.array(image)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    image = mindspore.Tensor(image)
    image = image.reshape((1, 1, 100, 100))
    return image



In [4]:
if __name__ == "__main__":
    """
    # 定义超参数
    batch_size = 400
    epoch = 100
    # 使用ascend加速
    context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")
    # 加载训练集
    dataset = load_dataset("./train", batch_size=batch_size)
    # 实例化网络
    net = my_net()
    # 定义损失函数
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    # 定义优化器
    net_opt = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)

    config_ck = CheckpointConfig(save_checkpoint_steps=32, keep_checkpoint_max=5)
    ckpt_cb = ModelCheckpoint(prefix='mynet', directory='./final_model', config=config_ck)
    model = mindspore.Model(net, loss_fn=net_loss, optimizer=net_opt, metrics={"Accuracy": nn.Accuracy()})
    print('Start training')
    model.train(epoch, train_dataset=dataset, callbacks=[ckpt_cb, LossMonitor()])
    print('finish training')
    
    # 加载预测test文件,生成txt
    # test数据集
    testset = load_testdata(testdata_path="./test/img_0.jpg")
    for i in range(1, 500):
        testdata_path = "./test/img_" + str(i) + ".jpg"
        image = load_testdata(testdata_path=testdata_path)
        op = ops.Concat(0)
        testset = op((testset, image))
    """
    # 加载训练好的网络并预测
    resnet = my_net()
    # 损失函数
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    # 将模型参数存入parameter的字典中
    param_dict = load_checkpoint("./final_model/mynet_5-100_12.ckpt")
    load_param_into_net(resnet, param_dict)
    model = mindspore.Model(resnet, net_loss, metrics={"accuracy"})
    temp = model.predict(testset)
    result = np.argmax(temp, axis=1)
    class_name = {0: "冰激凌", 1: "鸡蛋布丁", 2: "烤冷面", 3: "芒果班戟", 4: " 三明治", 5: "松鼠鱼",
                  6: "甜甜圈", 7: "土豆泥", 8: "小米粥", 9: "玉米饼"}
    # 写入预测结果
    with open("./final_result.txt", "w") as op:
        for i in range(500):
            data = result[i]
            op.write(str(data) + "\n")
        op.close()
    print("Finished all")
    

Start training
epoch: 1 step: 12, loss is 2.3026552200317383
epoch: 2 step: 12, loss is 2.302461862564087
epoch: 3 step: 12, loss is 2.3024673461914062
epoch: 4 step: 12, loss is 2.3023629188537598
epoch: 5 step: 12, loss is 2.3020920753479004
epoch: 6 step: 12, loss is 2.302269220352173
epoch: 7 step: 12, loss is 2.301931619644165
epoch: 8 step: 12, loss is 2.3013906478881836
epoch: 9 step: 12, loss is 2.2997443675994873
epoch: 10 step: 12, loss is 2.294198513031006
epoch: 11 step: 12, loss is 2.2917513847351074
epoch: 12 step: 12, loss is 2.284602642059326
epoch: 13 step: 12, loss is 2.2713851928710938
epoch: 14 step: 12, loss is 2.2323737144470215
epoch: 15 step: 12, loss is 2.2423315048217773
epoch: 16 step: 12, loss is 2.263306140899658
epoch: 17 step: 12, loss is 2.1865086555480957
epoch: 18 step: 12, loss is 2.1567740440368652
epoch: 19 step: 12, loss is 2.1192569732666016
epoch: 20 step: 12, loss is 2.344508409500122
epoch: 21 step: 12, loss is 2.2165355682373047
epoch: 22 step